# Creating a custom WPC Service adapter
This chapter is for those users, who wish not to use EL2GO, but rather have their own solution.

To help you getting started with creating a custom Service, we have a [Cookiecutter](https://cookiecutter.readthedocs.io/en/stable/) template located in: `<spsdk_repo>/examples/templates/cookiecutter-spsdk-wpc-service-plugin.zip`  
This template will create a Python project, containing a plugin for SPSDK

1. Install cookiecutter: `pip install cookiecutter`
1. Create your project: `cookiecutter <spsdk_repo>/examples/plugins/templates/cookiecutter-spsdk-wpc-service-plugin.zip`
    * follow the prompts
    * your project will be created in the current working directory
1. Class representing your adapter will be located in: `<your_project_name/your_project_name/your_project_name.py>`
1. Implement necessary methods
    * `get_validation_schemas`
        * Use this method to provide JSON validation schema for your class (parameters for the `__init__` method)
        * Schema will be used when instantiating your class using configuration data via `.load_from_config` method
        * This schema will be also used to generate configuration file using `nxpwpc get-template`
    * `get_wpc_cert`
        * This method is the bread and butter of your adapter
        * Here is where you'll implement the process of getting the WPC Certificate Chain
1. Install your plugin into virtual environment where you have SPSDK
    * `pip install (--editable) <path/to/your/project>`
    * The `--editable` flag is used for development purposes. It will allow you to develop your plugin without a need to re-install your plugin after every change
1. Verify your plugin was picked-up by SPSDK
    * `nxpwpc get-template --help`
    * You should see the name of your plugin in `--service-type` option
    * Name of the plugin is set during project creation, also defined in your class as `identifier` attribute

## Example

Commands below will create a new WPC Service Plugin Python project.

In [13]:
!pip install cookiecutter
!cookiecutter ../plugins/templates/cookiecutter-spsdk-wpc-service-plugin.zip --overwrite-if-exists --replay --replay-file cookiecutter-replay-file.json
!pip install -e my_wpc_service_plugin --no-deps

Looking in indexes: https://pypi.org/simple, https://nl2-nxrm.sw.nxp.com/repository/spsdk_pypi/simple/
  Using cached cookiecutter-2.6.0-py3-none-any.whl.metadata (7.3 kB)
Using cached cookiecutter-2.6.0-py3-none-any.whl (39 kB)
Looking in indexes: https://pypi.org/simple, https://nl2-nxrm.sw.nxp.com/repository/spsdk_pypi/simple/
Obtaining my_wpc_service_plugin
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for my_wpc_service_plugin (pyproject.toml): started
  Building editable for my_wpc_service_plugin (pyproject.toml): fini

In [14]:
!nxpwpc get-template --help


Help for nested command: 'get-template'
Command Hierarchy: nxpwpc ▶ get-template

──────────────────────── Root command (nxpwpc) help ────────────────────────

Options:
  -v, --verbose  Print more detailed information
  -vv, --debug   Display more debugging information.
  --version      Show the version and exit.
  --help         Show this message and exit.

─────────────────────── Command (get-template) help ───────────────────────
Generate a configuration template.
Usage: nxpwpc get-template [OPTIONS]

Options:
  -st, --service-type [el2go|mywpc]
                                  Name of the WPC service type.  [required]
  -tt, --target-type [mboot|model]
                                  Name of the MCU target type.  [required]
  -f, --family [mwct2015s|mwct2012|mwct2016s|mwct2014s..., and more. Use 'get-families' command to show all.]
                                  [required] Select the chip family.
  -r, --revision TEXT             Chip revision; if not specified, most recent


Please note the `mywpc` service type above

In [15]:
!nxpwpc get-template -st mywpc -tt model -f mwct2d17s -o config_with_plugin.yaml --force

The WPC template for mwct2d17s, Revision: latest has been saved into config_with_plugin.yaml YAML file


Please note the `service_parameters` section of [config_with_plugin.yaml](./config_with_plugin.yaml).  
Keys `param1` and `param2` are coming from `.get_validation_schemas` method from the plugin.  
When instantiating the plugin class using `.load_from_config`, these two keys will will be passed to the `__init__` method.